# tf.Variable() 与 tf.get_variable() 参数对比

## tf.Variable()

```Python
tf.Variable(initial_value=None, trainable=True, collections=None, validate_shape=True, 
caching_device=None, name=None, variable_def=None, dtype=None, expected_shape=None, 
import_scope=None)
```

## tf.get_variable()

```Python
tf.get_variable(name, shape=None, dtype=None, initializer=None, regularizer=None, 
trainable=True, collections=None, caching_device=None, partitioner=None, validate_shape=True, 
custom_getter=None)
```

# 区别

## 命名冲突的解决方式

使用`tf.Variable`时，如果检测到命名冲突，系统会自己处理。使用`tf.get_variable()`时，系统不会处理冲突，而会报错。

### tf.Variable()

In [1]:
import tensorflow as tf

w_1 = tf.Variable(3,name='w_1') 
w_2 = tf.Variable(1,name='w_1') 

# 这两个变量重名了，但实际结果如何呢？
print('w_1: ',w_1)
print('w_2: ',w_2)

Instructions for updating:
Colocations handled automatically by placer.
w_1:  <tf.Variable 'w_1:0' shape=() dtype=int32_ref>
w_2:  <tf.Variable 'w_1_1:0' shape=() dtype=int32_ref>


用`tf.Variable()`创建的变量，出现变量命名冲突，系统自己处理了，就是在变量名后面直接接一个`_1`

### tf.get_variable()

In [2]:
import tensorflow as tf

w_1 = tf.get_variable(name='w_1',initializer=1)
w_2 = tf.get_variable(name='w_1',initializer=2)

# 这两个变量重名了，但实际结果如何呢？
print('w_1: ',w_1)
print('w_2: ',w_2)

ValueError: Variable w_1 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1801, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3300, in create_op
    op_def=op_def)
  File "C:\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)


用`tf.get_variable()`创建的变量，如果命名冲突，则直接报错

## 创建的变量是否可重用

基于这两个函数的特性，当我们需要共享变量的时候，需要使用`tf.get_variable()`。在其他情况下，这两个的用法是一样的。

In [3]:
import tensorflow as tf

with tf.variable_scope('scope1'):
    w1 = tf.get_variable('w1',shape=[]) # 这是一个标量，0阶张量
    w2 = tf.Variable(0.0,name='w2')
    
with tf.variable_scope('scope1',reuse=True):
    w1_p = tf.get_variable('w1',shape=[]) # 重用变量命名空间scope1下的变量w1
    w2_p = tf.Variable(1.0,name='w2') # 重新创建了一个变量
    
print('w1: ',w1)
print('w2: ',w2)
print('w1_p: ',w1_p)
print('w2_p: ',w2_p)

w1:  <tf.Variable 'scope1/w1:0' shape=() dtype=float32_ref>
w2:  <tf.Variable 'scope1/w2:0' shape=() dtype=float32_ref>
w1_p:  <tf.Variable 'scope1/w1:0' shape=() dtype=float32_ref>
w2_p:  <tf.Variable 'scope1_1/w2:0' shape=() dtype=float32_ref>


**由于`tf.Variable()` 每次都在创建新对象，所有`reuse=True` 和它并没有什么关系**。对于`get_variable()`，来说，如果已经创建的变量对象，就把那个对象返回，如果没有创建变量对象的话，就创建一个新的。

## 初始化方式

### tf.Variable()

`tf.Variable()` 初始化是直接传入`initial_value` ， 我们使用的时候一般是这样子初始化的：

In [4]:
a = tf.Variable(initial_value=tf.random_normal(shape=[200, 100], stddev=0.1), trainable=True)
b = tf.Variable(initial_value=[1,2,3])

sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 只要是变量都需要这句话来执行初始化操作
print(sess.run(a))
print(sess.run(b))

[[ 0.0375093   0.16840783  0.1753603  ... -0.08809108 -0.04974153
   0.01019014]
 [ 0.01226684  0.03546741 -0.0826202  ... -0.0543442  -0.13129134
  -0.10323721]
 [-0.15615217 -0.03872107 -0.12579481 ... -0.30727357  0.1921245
   0.04172325]
 ...
 [ 0.0493133  -0.01106436  0.05944272 ... -0.14416675 -0.12706992
  -0.13809983]
 [-0.09015548  0.05215864  0.12855117 ...  0.14209282 -0.18500495
  -0.05972939]
 [-0.02720414 -0.04922058  0.08623477 ...  0.0244412   0.00779654
  -0.06041741]]
[1 2 3]


### tf.get_variable()

`tf.get_variable()`初始化是传入一个`initializer`,初始化器有很多，我们可以自己决定用哪一种。

In [5]:
c = tf.get_variable(name = 'weights', shape=[2, 5], dtype=tf.float32, initializer=tf.random_normal_initializer(stddev=0.1))

sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 只要是变量都需要这句话来执行初始化操作
print(sess.run(c))
sess.close()

[[-1.8793716e-04 -7.3708318e-02  2.0209624e-02 -1.6378945e-02
  -8.0871411e-02]
 [-9.2996784e-02  7.6715201e-02 -2.1203433e-01  1.7378502e-01
  -6.1321296e-02]]
